# Two-Factor Hull-White Model - GPU Simulation

This notebook runs the Two-Factor Hull-White interest rate model simulation on Google Colab with GPU acceleration.

**Important**: Enable GPU runtime before running!
- Go to: Runtime → Change runtime type → Hardware accelerator → GPU


## 1. Setup and Installation


In [ ]:
# Check GPU availability
!nvidia-smi


In [ ]:
# Install required packages
!pip install numpy scipy flask flask-cors requests -q

# Install CuPy for GPU acceleration (for CUDA 12.x)
# Check your CUDA version with nvidia-smi and adjust if needed
!pip install cupy-cuda12x -q


In [ ]:
# Verify CuPy installation
import cupy as cp
print(f"CuPy version: {cp.__version__}")
print(f"CUDA available: {cp.cuda.is_available()}")
if cp.cuda.is_available():
    print(f"GPU: {cp.cuda.Device().name}")
    mem = cp.cuda.Device().mem_info
    print(f"Memory: {mem[0]/1e9:.1f} GB free / {mem[1]/1e9:.1f} GB total")


## 2. Model Implementation


In [ ]:
import numpy as np
from dataclasses import dataclass, field
from typing import Optional
from scipy.interpolate import CubicSpline
import time
from datetime import datetime
import matplotlib.pyplot as plt

@dataclass
class HullWhite2FParams:
    """Parameters for the Two-Factor Hull-White model."""
    a: float = 0.1          # Mean reversion speed for r
    b: float = 0.04         # Mean reversion speed for u
    sigma1: float = 0.01    # Volatility of r
    sigma2: float = 0.008   # Volatility of u
    rho: float = -0.3       # Correlation between factors


class YieldCurve:
    """Represents an initial yield curve."""
    
    def __init__(self, maturities: np.ndarray, yields: np.ndarray):
        self.maturities = np.array(maturities)
        self.yields = np.array(yields)
        self._spline = CubicSpline(maturities, yields, bc_type='natural')
        self._compute_forward_rates()
    
    def _compute_forward_rates(self):
        fine_maturities = np.linspace(0.001, self.maturities[-1], 1000)
        yields_fine = self._spline(fine_maturities)
        dy_dt = self._spline(fine_maturities, 1)
        forward_rates = yields_fine + fine_maturities * dy_dt
        self._forward_spline = CubicSpline(fine_maturities, forward_rates, bc_type='natural')
    
    def yield_at(self, T):
        T = np.atleast_1d(T)
        return self._spline(np.clip(T, self.maturities[0], self.maturities[-1]))
    
    def forward_rate(self, T):
        T = np.atleast_1d(T)
        return self._forward_spline(np.clip(T, 0.001, self.maturities[-1]))
    
    def discount_factor(self, T):
        T = np.atleast_1d(T)
        return np.exp(-self.yield_at(T) * T)


def get_us_treasury_curve():
    """Get default US Treasury yield curve (Nov 2024 approximate rates)."""
    rates = {
        '1M': 0.0535, '2M': 0.0530, '3M': 0.0525, '4M': 0.0518,
        '6M': 0.0505, '1Y': 0.0475, '2Y': 0.0435, '3Y': 0.0420,
        '5Y': 0.0415, '7Y': 0.0420, '10Y': 0.0430, '20Y': 0.0465, '30Y': 0.0450
    }
    maturities_map = {
        '1M': 1/12, '2M': 2/12, '3M': 3/12, '4M': 4/12, '6M': 6/12,
        '1Y': 1, '2Y': 2, '3Y': 3, '5Y': 5, '7Y': 7, '10Y': 10, '20Y': 20, '30Y': 30
    }
    
    # Convert to continuous compounding
    continuous = {k: 2 * np.log(1 + v / 2) for k, v in rates.items()}
    
    maturities = [maturities_map[k] for k in rates.keys()]
    yields = [continuous[k] for k in rates.keys()]
    
    return YieldCurve(np.array(maturities), np.array(yields))


class HullWhite2FModel:
    """Two-Factor Hull-White Interest Rate Model - OPTIMIZED."""
    
    def __init__(self, params: HullWhite2FParams, initial_curve: YieldCurve):
        self.params = params
        self.initial_curve = initial_curve
        self._calibrate_theta()
    
    def _calibrate_theta(self):
        self.theta_grid = np.linspace(0.001, self.initial_curve.maturities[-1], 500)
        f = self.initial_curve.forward_rate(self.theta_grid)
        df_dt = np.gradient(f, self.theta_grid)
        a, sigma1 = self.params.a, self.params.sigma1
        self.theta_values = df_dt + a * f + (sigma1**2 / (2 * a)) * (1 - np.exp(-2 * a * self.theta_grid))
        self._theta_spline = CubicSpline(self.theta_grid, self.theta_values, bc_type='natural')
    
    def theta(self, t):
        t = np.atleast_1d(t)
        return self._theta_spline(np.clip(t, self.theta_grid[0], self.theta_grid[-1]))
    
    def initial_short_rate(self):
        return float(self.initial_curve.forward_rate(np.array([0.001]))[0])
    
    def precompute_bond_params(self, times, maturities):
        """Pre-compute all bond pricing parameters for vectorized computation."""
        a, b = self.params.a, self.params.b
        n_steps = len(times)
        n_mats = len(maturities)
        
        # Arrays to store precomputed values
        self._B1 = np.zeros((n_steps, n_mats))
        self._B2 = np.zeros((n_steps, n_mats))
        self._ln_A_base = np.zeros((n_steps, n_mats))
        
        for i, t in enumerate(times):
            for j, tau in enumerate(maturities):
                T = t + tau
                if tau > 0:
                    self._B1[i, j] = (1 - np.exp(-a * tau)) / a
                    self._B2[i, j] = (1 - np.exp(-b * tau)) / b
                    
                    P_0_T = self.initial_curve.discount_factor(np.array([T]))[0]
                    P_0_t = self.initial_curve.discount_factor(np.array([max(t, 0.001)]))[0]
                    f_0_t = self.initial_curve.forward_rate(np.array([max(t, 0.001)]))[0]
                    
                    self._ln_A_base[i, j] = np.log(P_0_T / P_0_t) - self._B1[i, j] * f_0_t
        
        self._maturities = maturities
        self._precomputed = True
    
    def yield_curves_vectorized(self, times, r_all, u_all, maturities):
        """
        VECTORIZED yield curve computation for all trials and time steps.
        
        Args:
            times: (n_steps+1,) array of time points
            r_all: (n_trials, n_steps+1) array of short rates
            u_all: (n_trials, n_steps+1) array of second factor
            maturities: (n_mats,) array of maturities
        
        Returns:
            (n_trials, n_steps+1, n_mats) array of yields
        """
        n_trials, n_steps_plus1 = r_all.shape
        n_mats = len(maturities)
        a, b = self.params.a, self.params.b
        
        # Pre-compute B factors and A terms for all (time, maturity) pairs
        # B1[step, mat], B2[step, mat], ln_A[step, mat]
        B1 = np.zeros((n_steps_plus1, n_mats))
        B2 = np.zeros((n_steps_plus1, n_mats))
        ln_A = np.zeros((n_steps_plus1, n_mats))
        
        for i, t in enumerate(times):
            for j, tau in enumerate(maturities):
                if tau > 0:
                    B1[i, j] = (1 - np.exp(-a * tau)) / a
                    B2[i, j] = (1 - np.exp(-b * tau)) / b
                    
                    T = t + tau
                    P_0_T = self.initial_curve.discount_factor(np.array([T]))[0]
                    P_0_t = self.initial_curve.discount_factor(np.array([max(t, 0.001)]))[0]
                    f_0_t = self.initial_curve.forward_rate(np.array([max(t, 0.001)]))[0]
                    
                    ln_A[i, j] = np.log(P_0_T / P_0_t) - B1[i, j] * f_0_t
        
        # Now compute yields using broadcasting
        # r_all: (n_trials, n_steps+1) -> (n_trials, n_steps+1, 1)
        # B1: (n_steps+1, n_mats) -> (1, n_steps+1, n_mats)
        r_expanded = r_all[:, :, np.newaxis]  # (n_trials, n_steps+1, 1)
        u_expanded = u_all[:, :, np.newaxis]  # (n_trials, n_steps+1, 1)
        
        B1_expanded = B1[np.newaxis, :, :]    # (1, n_steps+1, n_mats)
        B2_expanded = B2[np.newaxis, :, :]    # (1, n_steps+1, n_mats)
        ln_A_expanded = ln_A[np.newaxis, :, :]  # (1, n_steps+1, n_mats)
        
        # ln(P) = ln_A - B1*r - B2*u
        ln_P = ln_A_expanded - B1_expanded * r_expanded - B2_expanded * u_expanded
        
        # yield = -ln(P) / tau
        # maturities: (n_mats,) -> (1, 1, n_mats)
        tau_expanded = maturities[np.newaxis, np.newaxis, :]
        
        # Avoid division by zero for tau=0
        yields = np.where(tau_expanded > 0, -ln_P / tau_expanded, r_expanded)
        
        return yields

print("✓ Model classes defined (OPTIMIZED)")


## 3. Simulators (CPU & GPU)


In [ ]:
try:
    import cupy as cp
    GPU_AVAILABLE = cp.cuda.is_available()
    if GPU_AVAILABLE:
        print(f"✓ CuPy loaded, GPU: {cp.cuda.Device().name}")
except:
    cp = None
    GPU_AVAILABLE = False
    print("⚠ CuPy not available, will use CPU")

@dataclass
class SimulationResult:
    """Results from Monte Carlo simulation."""
    times: np.ndarray
    short_rates: np.ndarray
    second_factor: np.ndarray
    yield_curve_times: np.ndarray
    yield_curves: np.ndarray
    mean_short_rate: np.ndarray
    std_short_rate: np.ndarray
    mean_yield_curves: np.ndarray
    n_trials: int
    n_steps: int
    dt: float
    total_time: float
    execution_time: float
    device: str
    timestamp: str = field(default_factory=lambda: datetime.now().isoformat())


class HullWhite2FSimulator:
    """CPU-based Monte Carlo simulator - OPTIMIZED with vectorization."""
    
    def __init__(self, model: HullWhite2FModel):
        self.model = model
        self.params = model.params
    
    def simulate(self, n_trials=1000, n_steps=100, total_time=10.0, 
                 yield_maturities=None, seed=None):
        start_time = time.perf_counter()
        
        if seed is not None:
            np.random.seed(seed)
        
        if yield_maturities is None:
            yield_maturities = np.array([0.25, 0.5, 1, 2, 3, 5, 7, 10, 20, 30])
        
        dt = total_time / n_steps
        sqrt_dt = np.sqrt(dt)
        a, b = self.params.a, self.params.b
        sigma1, sigma2 = self.params.sigma1, self.params.sigma2
        rho = self.params.rho
        
        times = np.linspace(0, total_time, n_steps + 1)
        
        # Pre-compute theta values for all time steps
        theta_all = self.model.theta(times[:-1])
        
        # Initialize paths
        r = np.zeros((n_trials, n_steps + 1), dtype=np.float64)
        u = np.zeros((n_trials, n_steps + 1), dtype=np.float64)
        r[:, 0] = self.model.initial_short_rate()
        
        # Generate all random numbers at once
        Z1 = np.random.standard_normal((n_trials, n_steps))
        Z_indep = np.random.standard_normal((n_trials, n_steps))
        Z2 = rho * Z1 + np.sqrt(1 - rho**2) * Z_indep
        
        # Simulate paths (vectorized across trials)
        for i in range(n_steps):
            r[:, i + 1] = r[:, i] + (theta_all[i] + u[:, i] - a * r[:, i]) * dt + sigma1 * sqrt_dt * Z1[:, i]
            u[:, i + 1] = u[:, i] - b * u[:, i] * dt + sigma2 * sqrt_dt * Z2[:, i]
        
        # VECTORIZED yield curve computation
        yield_curves = self.model.yield_curves_vectorized(times, r, u, yield_maturities)
        
        execution_time = time.perf_counter() - start_time
        
        return SimulationResult(
            times=times, short_rates=r, second_factor=u,
            yield_curve_times=yield_maturities, yield_curves=yield_curves,
            mean_short_rate=np.mean(r, axis=0), std_short_rate=np.std(r, axis=0),
            mean_yield_curves=np.mean(yield_curves, axis=0),
            n_trials=n_trials, n_steps=n_steps, dt=dt, total_time=total_time,
            execution_time=execution_time, device='cpu'
        )


class HullWhite2FSimulatorGPU:
    """GPU-accelerated Monte Carlo simulator - FULLY OPTIMIZED."""
    
    def __init__(self, model: HullWhite2FModel, force_cpu=False):
        self.model = model
        self.params = model.params
        self.use_gpu = GPU_AVAILABLE and not force_cpu
        self.xp = cp if self.use_gpu else np
    
    def simulate(self, n_trials=1000, n_steps=100, total_time=10.0,
                 yield_maturities=None, seed=None):
        start_time = time.perf_counter()
        xp = self.xp
        
        if seed is not None:
            if self.use_gpu:
                cp.random.seed(seed)
            else:
                np.random.seed(seed)
        
        if yield_maturities is None:
            yield_maturities = np.array([0.25, 0.5, 1, 2, 3, 5, 7, 10, 20, 30])
        
        dt = total_time / n_steps
        sqrt_dt = float(np.sqrt(dt))
        a, b = self.params.a, self.params.b
        sigma1, sigma2 = self.params.sigma1, self.params.sigma2
        rho = self.params.rho
        rho_complement = float(np.sqrt(1 - rho**2))
        
        times = np.linspace(0, total_time, n_steps + 1)
        
        # Pre-compute theta values on CPU, then transfer to GPU
        theta_cpu = self.model.theta(times[:-1]).astype(np.float32)
        
        if self.use_gpu:
            theta_gpu = cp.asarray(theta_cpu)
        else:
            theta_gpu = theta_cpu
        
        # Initialize paths on GPU
        r = xp.zeros((n_trials, n_steps + 1), dtype=xp.float32)
        u = xp.zeros((n_trials, n_steps + 1), dtype=xp.float32)
        r[:, 0] = float(self.model.initial_short_rate())
        
        # Generate all random numbers at once on GPU (much faster)
        if self.use_gpu:
            Z1 = xp.random.standard_normal((n_trials, n_steps), dtype=xp.float32)
            Z_indep = xp.random.standard_normal((n_trials, n_steps), dtype=xp.float32)
        else:
            Z1 = xp.random.standard_normal((n_trials, n_steps)).astype(xp.float32)
            Z_indep = xp.random.standard_normal((n_trials, n_steps)).astype(xp.float32)
        Z2 = rho * Z1 + rho_complement * Z_indep
        
        # Pre-compute constants
        dt_f = float(dt)
        a_f, b_f = float(a), float(b)
        s1_sqrt_dt = float(sigma1 * sqrt_dt)
        s2_sqrt_dt = float(sigma2 * sqrt_dt)
        
        # Simulate paths - fully vectorized across trials
        for i in range(n_steps):
            theta_t = theta_gpu[i]
            r[:, i + 1] = r[:, i] + (theta_t + u[:, i] - a_f * r[:, i]) * dt_f + s1_sqrt_dt * Z1[:, i]
            u[:, i + 1] = u[:, i] - b_f * u[:, i] * dt_f + s2_sqrt_dt * Z2[:, i]
        
        # Synchronize and transfer to CPU
        if self.use_gpu:
            cp.cuda.Stream.null.synchronize()
            r_cpu = cp.asnumpy(r).astype(np.float64)
            u_cpu = cp.asnumpy(u).astype(np.float64)
        else:
            r_cpu = r.astype(np.float64)
            u_cpu = u.astype(np.float64)
        
        # VECTORIZED yield curve computation on CPU (already optimized)
        yield_curves = self.model.yield_curves_vectorized(times, r_cpu, u_cpu, yield_maturities)
        
        execution_time = time.perf_counter() - start_time
        
        if self.use_gpu:
            device_name = f"gpu ({cp.cuda.Device().name})"
        else:
            device_name = "cpu"
        
        return SimulationResult(
            times=times, short_rates=r_cpu,
            second_factor=u_cpu,
            yield_curve_times=yield_maturities, yield_curves=yield_curves,
            mean_short_rate=np.mean(r_cpu, axis=0),
            std_short_rate=np.std(r_cpu, axis=0),
            mean_yield_curves=np.mean(yield_curves, axis=0),
            n_trials=n_trials, n_steps=n_steps, dt=dt, total_time=total_time,
            execution_time=execution_time, device=device_name
        )

print(f"✓ Simulators defined (OPTIMIZED)")
print(f"  GPU available: {GPU_AVAILABLE}")


## 4. Configure and Run Simulation

Adjust the parameters below to customize your simulation:


In [ ]:
# ============================================
# USER-CONFIGURABLE PARAMETERS
# ============================================
N_TRIALS = 5000      # Number of Monte Carlo trials (10 - 100,000)
N_STEPS = 100        # Number of time steps (5 - 500)
TOTAL_TIME = 10.0    # Simulation horizon in years
SEED = 42            # Random seed for reproducibility

# Model parameters (Two-Factor Hull-White)
params = HullWhite2FParams(
    a=0.10,       # Mean reversion for short rate (~10 year half-life)
    b=0.04,       # Mean reversion for second factor
    sigma1=0.01,  # Short rate volatility (100 bps/year)
    sigma2=0.008, # Second factor volatility (80 bps/year)
    rho=-0.30     # Factor correlation (negative = curve flattening when rates rise)
)
# ============================================

# Create model calibrated to US Treasury curve
curve = get_us_treasury_curve()
model = HullWhite2FModel(params, curve)

print(f"✓ Model created")
print(f"  Initial short rate: {model.initial_short_rate()*100:.2f}%")
print(f"\n📊 Simulation settings:")
print(f"  Trials: {N_TRIALS:,}")
print(f"  Steps: {N_STEPS}")
print(f"  Horizon: {TOTAL_TIME} years")
print(f"  dt: {TOTAL_TIME/N_STEPS:.4f} years")


In [ ]:
# Run CPU simulation
print("🖥️  Running CPU simulation...")
cpu_sim = HullWhite2FSimulator(model)
cpu_result = cpu_sim.simulate(n_trials=N_TRIALS, n_steps=N_STEPS, total_time=TOTAL_TIME, seed=SEED)
print(f"  ✓ Completed in {cpu_result.execution_time:.4f} seconds")

# Run GPU simulation
print("\n🎮 Running GPU simulation...")
gpu_sim = HullWhite2FSimulatorGPU(model)
gpu_result = gpu_sim.simulate(n_trials=N_TRIALS, n_steps=N_STEPS, total_time=TOTAL_TIME, seed=SEED)
print(f"  ✓ Completed in {gpu_result.execution_time:.4f} seconds")

# Calculate speedup
speedup = cpu_result.execution_time / gpu_result.execution_time

print("\n" + "="*60)
print("📈 PERFORMANCE COMPARISON")
print("="*60)
print(f"\n{'Device':<20} {'Time (s)':<15} {'Trials/sec':<15}")
print("-"*50)
print(f"{'CPU':<20} {cpu_result.execution_time:<15.4f} {N_TRIALS/cpu_result.execution_time:<15,.0f}")
print(f"{'GPU':<20} {gpu_result.execution_time:<15.4f} {N_TRIALS/gpu_result.execution_time:<15,.0f}")
print("-"*50)
print(f"\n🚀 GPU SPEEDUP: {speedup:.2f}x")
print("="*60)


## 5. Visualization


## 6. Interactive Yield Curve Explorer

Use the slider to see how the yield curve evolves through time:


In [ ]:
# Create comprehensive visualization
plt.style.use('dark_background')
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Two-Factor Hull-White Model Simulation Results', fontsize=16, fontweight='bold', color='white')

result = gpu_result  # Use GPU results for visualization

# 1. Initial yield curve
ax1 = axes[0, 0]
maturities = result.yield_curve_times
initial_yields = result.mean_yield_curves[0] * 100
ax1.plot(maturities, initial_yields, 'o-', color='#00d9a5', linewidth=2, markersize=6)
ax1.fill_between(maturities, 0, initial_yields, alpha=0.2, color='#00d9a5')
ax1.set_xlabel('Maturity (Years)')
ax1.set_ylabel('Yield (%)')
ax1.set_title('Initial US Treasury Yield Curve', fontweight='bold')
ax1.grid(alpha=0.2)

# 2. Short rate paths
ax2 = axes[0, 1]
times = result.times
mean_rate = result.mean_short_rate * 100
std_rate = result.std_short_rate * 100

# Plot sample paths
for i in range(min(30, result.n_trials)):
    ax2.plot(times, result.short_rates[i] * 100, alpha=0.08, color='#00b4d8', linewidth=0.5)

ax2.fill_between(times, mean_rate - std_rate, mean_rate + std_rate, alpha=0.3, color='#00d9a5')
ax2.plot(times, mean_rate, color='#00d9a5', linewidth=2, label='Mean ± 1σ')
ax2.set_xlabel('Time (Years)')
ax2.set_ylabel('Short Rate (%)')
ax2.set_title(f'Short Rate Simulation ({result.n_trials:,} paths)', fontweight='bold')
ax2.legend()
ax2.grid(alpha=0.2)

# 3. Yield curve evolution
ax3 = axes[1, 0]
n_curves = 8
step_size = max(1, result.n_steps // (n_curves - 1))
for i, step in enumerate(range(0, result.n_steps + 1, step_size)):
    if step >= len(result.mean_yield_curves):
        step = len(result.mean_yield_curves) - 1
    t = result.times[step]
    yields = result.mean_yield_curves[step] * 100
    color = plt.cm.viridis(i / n_curves)
    ax3.plot(maturities, yields, '-', color=color, linewidth=2, label=f't={t:.1f}y')

ax3.set_xlabel('Maturity (Years)')
ax3.set_ylabel('Yield (%)')
ax3.set_title('Yield Curve Evolution Over Time', fontweight='bold')
ax3.legend(loc='upper right', fontsize=8, ncol=2)
ax3.grid(alpha=0.2)

# 4. Performance comparison bar chart
ax4 = axes[1, 1]
devices = ['CPU', 'GPU']
times_exec = [cpu_result.execution_time, gpu_result.execution_time]
colors = ['#00b4d8', '#00d9a5']
bars = ax4.bar(devices, times_exec, color=colors, width=0.5, edgecolor='white', linewidth=1.5)
ax4.set_ylabel('Execution Time (seconds)')
ax4.set_title(f'CPU vs GPU Performance (Speedup: {speedup:.1f}x)', fontweight='bold')

for bar, t in zip(bars, times_exec):
    ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
             f'{t:.3f}s', ha='center', va='bottom', fontsize=12, fontweight='bold')

ax4.set_ylim(0, max(times_exec) * 1.25)
ax4.grid(alpha=0.2, axis='y')

plt.tight_layout()
plt.savefig('simulation_results.png', dpi=150, bbox_inches='tight', facecolor='#0a0e14')
plt.show()

print("\n📁 Results saved to 'simulation_results.png'")


In [ ]:
from ipywidgets import interact, IntSlider

def plot_yield_curve_at_time(step):
    """Plot yield curve at a specific time step."""
    fig, ax = plt.subplots(figsize=(10, 5))
    fig.patch.set_facecolor('#0a0e14')
    ax.set_facecolor('#0a0e14')
    
    t = gpu_result.times[step]
    yields = gpu_result.mean_yield_curves[step] * 100
    initial = gpu_result.mean_yield_curves[0] * 100
    
    # Plot initial curve as reference
    ax.plot(maturities, initial, '--', color='gray', linewidth=1.5, alpha=0.7, label='Initial (t=0)')
    
    # Plot current curve
    ax.plot(maturities, yields, 'o-', color='#00d9a5', linewidth=2.5, markersize=8, label=f't = {t:.2f} years')
    ax.fill_between(maturities, initial, yields, alpha=0.15, color='#00d9a5')
    
    # Styling
    ax.set_xlabel('Maturity (Years)', fontsize=12, color='white')
    ax.set_ylabel('Yield (%)', fontsize=12, color='white')
    ax.set_title(f'Yield Curve at t = {t:.2f} years', fontsize=14, fontweight='bold', color='white')
    ax.legend(facecolor='#1a222d', edgecolor='gray', labelcolor='white')
    ax.grid(alpha=0.2)
    ax.tick_params(colors='white')
    for spine in ax.spines.values():
        spine.set_color('gray')
    
    plt.tight_layout()
    plt.show()

interact(
    plot_yield_curve_at_time,
    step=IntSlider(min=0, max=gpu_result.n_steps, step=1, value=0, 
                   description='Time Step:', style={'description_width': 'initial'})
)


## 7. Run Log and Export


In [ ]:
import json

# Generate comprehensive run log
run_log = {
    'timestamp': datetime.now().isoformat(),
    'configuration': {
        'n_trials': N_TRIALS,
        'n_steps': N_STEPS,
        'total_time': TOTAL_TIME,
        'dt': TOTAL_TIME / N_STEPS,
        'seed': SEED,
        'model_params': {
            'a': params.a,
            'b': params.b,
            'sigma1': params.sigma1,
            'sigma2': params.sigma2,
            'rho': params.rho
        }
    },
    'results': {
        'cpu': {
            'device': cpu_result.device,
            'execution_time_seconds': round(cpu_result.execution_time, 6),
            'paths_per_second': round(N_TRIALS / cpu_result.execution_time, 2),
            'final_mean_rate_pct': round(float(cpu_result.mean_short_rate[-1]) * 100, 4),
            'final_std_rate_pct': round(float(cpu_result.std_short_rate[-1]) * 100, 4)
        },
        'gpu': {
            'device': gpu_result.device,
            'execution_time_seconds': round(gpu_result.execution_time, 6),
            'paths_per_second': round(N_TRIALS / gpu_result.execution_time, 2),
            'final_mean_rate_pct': round(float(gpu_result.mean_short_rate[-1]) * 100, 4),
            'final_std_rate_pct': round(float(gpu_result.std_short_rate[-1]) * 100, 4)
        },
        'speedup': round(speedup, 2)
    }
}

print("="*60)
print("📋 RUN LOG")
print("="*60)
print(json.dumps(run_log, indent=2))
print("="*60)

# Save to file
with open('simulation_log.json', 'w') as f:
    json.dump(run_log, f, indent=2)
print("\n✓ Log saved to 'simulation_log.json'")
